## Data Import and Preprocessing

In [1]:
import pandas as pd
from chembl_webresource_client.new_client import new_client
import tk
import easygui

Input box will likely be used in future

In [2]:
# condition = easygui.enterbox('What condition are you interested in')

Target is selected and searched within the Chembl db before being turned into a dataframe from dict

In [3]:
condition='coronavirus'
target = new_client.target
target_query=target.search(condition)
targets=pd.DataFrame.from_dict(target_query)
targets

,cross_references,organism,pref_name,score,species_group_flag,target_chembl_id,target_components,target_type,tax_id
0,[],Coronavirus,Coronavirus,17.0,False,CHEMBL613732,[],ORGANISM,11119
1,[],SARS coronavirus,SARS coronavirus,15.0,False,CHEMBL612575,[],ORGANISM,227859
2,[],Feline coronavirus,Feline coronavirus,15.0,False,CHEMBL612744,[],ORGANISM,12663
3,[],Human coronavirus 229E,Human coronavirus 229E,13.0,False,CHEMBL613837,[],ORGANISM,11137
4,"[{'xref_id': 'P0C6U8', 'xref_name': None, 'xre...",SARS coronavirus,SARS coronavirus 3C-like proteinase,10.0,False,CHEMBL3927,"[{'accession': 'P0C6U8', 'component_descriptio...",SINGLE PROTEIN,227859
5,[],Middle East respiratory syndrome-related coron...,Middle East respiratory syndrome-related coron...,9.0,False,CHEMBL4296578,[],ORGANISM,1335626
6,"[{'xref_id': 'P0C6X7', 'xref_name': None, 'xre...",SARS coronavirus,Replicase polyprotein 1ab,4.0,False,CHEMBL5118,"[{'accession': 'P0C6X7', 'component_descriptio...",SINGLE PROTEIN,227859
7,[],Severe acute respiratory syndrome coronavirus 2,Replicase polyprotein 1ab,4.0,False,CHEMBL4523582,"[{'accession': 'P0DTD1', 'component_descriptio...",SINGLE PROTEIN,2697049


Target row is subsetted from dataframe and it's chembl id is extracted. Any target can be chosen and searched

In [4]:
selected_target=targets["target_chembl_id"][4]
selected_target

'CHEMBL3927'

Now using the target chembl id the database is searched for activity data but only returning results that have activity using IC50.

In [5]:
activity=new_client.activity
act_query=activity.filter(target_chembl_id=selected_target).filter(standard_type="IC50")
df=pd.DataFrame.from_dict(act_query)

Raw query results are saved to CSV

In [6]:
df.to_csv(condition + '_' + selected_target + '_' + 'query_raw.csv', index=False)

In [7]:
df=df[["canonical_smiles", "molecule_chembl_id", "standard_value"]]

In [8]:
df=df[df["standard_value"].notna()]

In [9]:
df["standard_value"] = df["standard_value"].astype("float")
df.dtypes

canonical_smiles       object
molecule_chembl_id     object
standard_value        float64
dtype: object

In [10]:
act_class = []
for i in df["standard_value"]:
    if float(i) >=10000:
        act_class.append("inactive")
    elif float(i) <= 1000:
        act_class.append("active")
    else:
        act_class.append("intermediate")

In [11]:
df["act_class"]=act_class

In [12]:
df.to_csv(condition + '_' + selected_target + '_' + 'preprocessed.csv', index=False)